<a href="https://colab.research.google.com/github/frizuma3/21013159_DataAnalytics/blob/main/Tutorial4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/1122131uhi/dataAnalytics/master/tutorial2dnndata.csv', index_col=0)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.9.2


In [ ]:
# make sure we have our data by printing it out
df[:5]
# df #all

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,...,Mon,Sat,Sun,Thu,Tue,Wed,year,temp,wdsp,NUM_TRIPS
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2009,20.0,7.4,0.631126
2,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,2009,30.0,10.8,0.724951
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,2009,30.4,5.7,0.716044
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,2009,37.9,16.4,0.799994
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2009,25.7,13.8,0.872611


In [ ]:
dnn_input_data = [df["year"], df["temp"], df["wdsp"], df["Sat"], df["Sun"], df["Mon"], df["Tue"], df["Wed"], df["Thu"], df["Fri"], df["Jan"], df["Feb"], df["Mar"], df["Apr"], df["May"], df["Jun"], df["Jul"], df["Aug"], df["Sep"], df["Oct"], df["Nov"], df["Dec"], df["NUM_TRIPS"]]
headers = ["year","temp", "wdsp", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","NUM_TRIPS"]
df_dnn_input = pd.concat(dnn_input_data, axis=1, keys=headers)
df_dnn_input.head()

,year,temp,wdsp,Sat,Sun,Mon,Tue,Wed,Thu,Fri,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,NUM_TRIPS
1,2009,20.0,7.4,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.631126
2,2009,30.0,10.8,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.724951
3,2009,30.4,5.7,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.716044
4,2009,37.9,16.4,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.799994
5,2009,25.7,13.8,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.872611


In [ ]:
training_dataset = df_dnn_input.sample(frac=0.8, random_state=0)
test_dataset = df_dnn_input.drop(training_dataset.index)

In [ ]:
training_features = training_dataset.copy()
test_features = test_dataset.copy()

training_labels = training_features.pop('NUM_TRIPS')
test_labels = test_features.pop('NUM_TRIPS')

In [ ]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [ ]:
training_labels = training_labels/SCALE_NUM_TRIPS
test_labels = test_labels/SCALE_NUM_TRIPS

In [ ]:
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [ ]:
# This is the only difference, instead of a single layer, we have our normalisation layer (22 inputs), 2 layers of 48, with 1 output. The 48 can be adjusted to improve the net.
dnn_model_1 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_1.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

In [ ]:
%%time
history = dnn_model_1.fit(
    training_features,
    training_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 8.31 s, sys: 457 ms, total: 8.77 s
Wall time: 8.17 s


In [ ]:
# remember, we want to minimise this. The model with the lowest is the best.
dnn_model_1_results = dnn_model_1.evaluate(test_features, test_labels, verbose=0)
print(dnn_model_1_results)

0.048479896038770676


In [ ]:
# make sure the labels match up with the dataframe from earlier.
input_1 = pd.DataFrame.from_dict(data =
				{
         'year' : [2009,2009,2009],
         'temp' : [61.8, 31.2, 40.0],
         'wdsp' : [5.0, 3.0, 8.0],
         'Sat' : [0,0,0],
         'Sun' : [0,0,0],
         'Mon' : [1,1,1],
         'Tue' : [0,0,0],
         'Wed' : [0,0,0],
         'Thu' : [0,0,0],
         'Fri' : [0,0,0],
         'Jan' : [0,0,0],
         'Feb' : [0,0,1],
         'Mar' : [0,0,0],
         'Apr' : [0,0,0],
         'May' : [0,0,0],
         'Jun' : [0,0,0],
         'Jul' : [0,0,0],
         'Aug' : [1,0,0],
         'Sep' : [0,0,0],
         'Oct' : [0,0,0],
         'Nov' : [0,0,0],
         'Dec' : [0,1,0],
        })


In [ ]:
linear_day_predictions = dnn_model_1.predict(input_1[:3])*SCALE_NUM_TRIPS
linear_day_predictions

1/1 [==============================] - 0s 83ms/step


array([[0.6667917 ],
       [0.78903544],
       [0.70978045]], dtype=float32)

For the first, you can see below, we are in August, Saturday (we have a monday), 2009. We have a lower temperature and lower windspeed. The difference in day is likely to account for the higher actual number of trips.

205	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	2009	64.9	4.2	0.700748221591537

For second (December):

337	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	35.2	6.6	0.62902717790116

For third (February):

28	0	0	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	2009	38.1	3.6	0.675798566550843

The results overall look reasonable. You can look into this further.


Considerations for Assignment. I would likely use another method to either standardise or normalise num_trips. And perhaps not bother and scale them in here with SCALE_NUM_TRIPS with a generally large number.

Obviously, you will likely want to have a number of variations but there is no reason you can't use most of the data given. Remember, the DNN is trying to solve a complex relationship, not a linear one.

Other things to consider when doing this would be to take validation. Of course, in the assignment you will have a much larger range of data i.e. from some date x to y.

This will give you more data. Taking real validation data that hasn't been shown to the model will give real results for you to check against rather than what I have done here (which is just for information).